In [4]:
!pip install pyportfolioopt

In [5]:
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
import pandas as pd
import cvxpy as cp


In [6]:
df = pd.read_csv('/Users/filipporonzino/Downloads/definitive_dataset.csv')


In [7]:
df['year'] = pd.DatetimeIndex(df['date']).year

In [8]:
year1 = 2015
year2 = 2016
permno_year1 = df[df['year'] == year1]['permno'].unique()
permno_year2 = df[df['year'] == year2]['permno'].unique()

In [9]:
print(permno_year1, permno_year2)

[10104 10107 10138 10145 10147 10299 10516 10696 10909 11308 11403 11404
 11533 11552 11600 11618 11674 11703 11786 11850 11891 11896 11955 12052
 12060 12062 12073 12308 12369 12448 12449 12476 12490 12558 12570 12622
 12623 12650 12872 12981 13035 13141 13168 13210 13323 13356 13407 13447
 13511 13567 13598 13641 13688 13721 13788 13856 13901 13911 13928 13936
 13949 13963 13983 14008 14295 14338 14541 14579 14593 14601 14702 15069
 15202 15456 15553 15560 15579 15720 16432 16600 16678 17005 17144 17279
 17478 17743 17750 17830 18163 18403 18411 18542 18729 19166 19350 19393
 19502 19561 20415 20482 20626 21020 21178 21186 21207 21371 21573 21776
 21792 21936 22103 22111 22293 22509 22517 22592 22752 22779 23026 23085
 23229 23393 23473 23501 23536 23579 23660 23712 23819 23887 23931 24010
 24053 24109 24205 24328 24459 24643 24766 24942 24985 25081 25099 25232
 25320 25419 25582 25778 25785 25953 26403 26710 26825 27043 27422 27633
 27756 27828 27887 27959 27983 27991 28222 28345 28

In [10]:
print(df.columns)

Index(['date', 'permno', 'stat_divyeld', '12_month_return', '3_month_return',
       'fed_funds_adj_close', 'fed_funds_volume', 'CAPEI', 'bm', 'evm',
       'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf', 'npm',
       'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce',
       'aftret_eq', 'aftret_invcapx', 'aftret_equity', 'GProf',
       'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio',
       'cash_lt', 'debt_at', 'debt_ebitda', 'short_debt', 'lt_debt',
       'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
       'debt_capital', 'de_ratio', 'at_turn', 'rect_turn', 'pay_turn',
       'sale_invcap', 'sale_equity', 'rd_sale', 'adv_sale', 'staff_sale',
       'accrual', 'ptb', 'divyield', 'prc', 'vol', 'ret', 'retx', 'mktcap',
       'prc_adj', 'naics_processed', 'ret_industry_tot',
       'ret_industry_relative', 'MACD_index', 'rsi', 'target', 'year'],
      dtype='object')


In [11]:
# Create an empty DataFrame to store the results
result_df = pd.DataFrame()
permno_1=21936
permno_2=22103
permno_3=22293
# List of permnos
permno_list = [permno_1, permno_2, permno_3]

# Loop through permnos
for permno in permno_list:
    # Filter DataFrame for each permno
    df_permno = df[df['permno'] == permno]
    
    # Select columns 'date' and 'prc'
    df_permno = df_permno[['date', 'prc']]
    
    # Rename columns
    df_permno.columns = ['date', permno]
    
    # Merge with result_df
    if result_df.empty:
        result_df = df_permno
    else:
        result_df = pd.merge(result_df, df_permno, on='date', how='outer')

df_allpermno = result_df.set_index('date')

# Display the result
print(df_allpermno)


                21936      22103  22293
date                                   
2008-01-02  14.481735  43.317329  23.37
2008-01-03  14.698492  41.843945  23.13
2008-01-04  14.963614  42.653318  22.44
2008-01-07  14.576171  40.674510  22.16
2008-01-08  15.015011  40.799034  21.68
...               ...        ...    ...
2023-12-22  18.445369  73.448750  30.23
2023-12-26  18.440571  74.060677  30.45
2023-12-27  18.398082  75.015066  30.56
2023-12-28  18.567955  74.956212  30.63
2023-12-29  18.680251  74.364160  30.45

[4024 rows x 3 columns]


In [12]:
import pypfopt as pfo

# Calculate mean historical returns using pypfopt
expected_returns = pfo.expected_returns.mean_historical_return(df_allpermno)
print(expected_returns)


21936    0.016075
22103    0.034431
22293    0.016715
dtype: float64


In [13]:
from scipy.stats import gaussian_kde

# Dictionary to store KDE distributions for each permno
kde_distributions = {}

# Loop through permnos
for permno_val in permno_list:
    # Filter DataFrame for each permno
    filtered_df = df[df['permno'] == permno_val]
    filtered_df = filtered_df[['permno', 'ret']]
    
    # Calculate KDE for returns
    kde = gaussian_kde(filtered_df['ret'])
    kde_distributions[permno_val] = kde
 

In [19]:
# Define the variables
w = cp.Variable(len(permno_list))  # Weights for distributions

x_values = np.linspace(filtered_df['ret'].min(), filtered_df['ret'].max(), 1000)
kde_permno_values = {}

for permno_val in permno_list:
    # Get the KDE distribution for the current permno
    kde_permno = kde_distributions[permno_val]
    
    # Evaluate the KDE values for the current permno
    kde_values = kde_permno.evaluate(x_values)
    
    # Store the KDE values in the dictionary
    kde_permno_values[permno_val] = kde_values

def l1_regularizer(w):
    return cp.norm(w, 1)



# Define the objective function
expected_return = cp.sum(cp.multiply(w, expected_returns))
entropies = [cp.sum(cp.entr(cp.multiply(w[i], kde_permno_values[permno_list[i]]))) for i in range(len(permno_list))]
lambdas = [0.00001] * len(permno_list)  # Adjust lambda parameters based on risk preference
weighted_entropies = [lambdas[i] * entropies[i] for i in range(len(permno_list))]
objective = cp.Maximize(expected_return + cp.sum(weighted_entropies))

# Define the constraints
constraints = [cp.sum(w) == 1, w >= 0]  # Ensure weights sum to 1 and are non-negative

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Get the optimal solution
optimal_weights = w.value
print(optimal_weights)

[0.2216824  0.46849678 0.30982083]


In [16]:
w0 = optimal_weights
# Now do again otimization assuming you have 

[0.22168237 0.4684968  0.30982083]
